In [ ]:
!pip install kaggle


## Multiclass Classification of TrashNet dataset

In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json here


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"yoonthiriaung","key":"8951ff53ffc460e54b754ab7566854a2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d feyzazkefe/trashnet
!unzip trashnet.zip -d ./trashnet


Dataset URL: https://www.kaggle.com/datasets/feyzazkefe/trashnet
License(s): unknown
  0% 0.00/40.8M [00:00<?, ?B/s]
100% 40.8M/40.8M [00:00<00:00, 1.31GB/s]
Archive:  trashnet.zip
  inflating: ./trashnet/dataset-resized/cardboard/cardboard1.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard10.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard100.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard101.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard102.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard103.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard104.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard105.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard106.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard107.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard108.jpg  
  inflating: ./trashnet/dataset-resized/cardboard/cardboard109.

In [ ]:
!kaggle datasets download -d mostafaabla/garbage-classification
!unzip garbage-classification.zip -d ./garbage


Streaming output truncated to the last 5000 lines.
  inflating: ./garbage/garbage_classification/paper/paper381.jpg  
  inflating: ./garbage/garbage_classification/paper/paper382.jpg  
  inflating: ./garbage/garbage_classification/paper/paper383.jpg  
  inflating: ./garbage/garbage_classification/paper/paper384.jpg  
  inflating: ./garbage/garbage_classification/paper/paper385.jpg  
  inflating: ./garbage/garbage_classification/paper/paper386.jpg  
  inflating: ./garbage/garbage_classification/paper/paper387.jpg  
  inflating: ./garbage/garbage_classification/paper/paper388.jpg  
  inflating: ./garbage/garbage_classification/paper/paper389.jpg  
  inflating: ./garbage/garbage_classification/paper/paper39.jpg  
  inflating: ./garbage/garbage_classification/paper/paper390.jpg  
  inflating: ./garbage/garbage_classification/paper/paper391.jpg  
  inflating: ./garbage/garbage_classification/paper/paper392.jpg  
  inflating: ./garbage/garbage_classification/paper/paper393.jpg  
  inflating:

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,random_split,WeightedRandomSampler

import os
import shutil
from pathlib import Path

In [ ]:
garbage_path = Path('./garbage/garbage_classification/')
trashnet_path = Path('./trashnet/dataset-resized/')
merged_path = Path('./merged_dataset/')

In [ ]:
# Final classes
classes = ['metal', 'paper', 'plastic', 'cardboard', 'trash', 'glass', 'biological', 'shoes', 'battery', 'clothes']

# Create merged dataset folders
for cls in classes:
    (merged_path / cls).mkdir(parents=True, exist_ok=True)

# Map Garbage glass classes to 'glass'
garbage_class_map = {
    'white-glass': 'glass',
    'brown-glass': 'glass',
    'green-glass': 'glass'
}


In [ ]:
def copy_images(src_path, class_map=None, include_classes=None):
    for class_name in os.listdir(src_path):
        src_class = Path(src_path) / class_name
        if not src_class.is_dir():
            continue

        # Only include desired classes
        if include_classes and class_name not in include_classes and (not class_map or class_name not in class_map):
            continue

        target_class = class_map[class_name] if class_map and class_name in class_map else class_name

        for img_file in src_class.iterdir():
            if img_file.is_file():
                shutil.copy(img_file, merged_path / target_class / img_file.name)

# Step 1: Copy Garbage dataset (merge glass)
garbage_classes_to_take = [
    'metal', 'paper', 'plastic', 'biological', 'cardboard', 'trash',
    'shoes', 'battery', 'clothes', 'white-glass', 'brown-glass', 'green-glass'
]
copy_images(garbage_path, class_map=garbage_class_map, include_classes=garbage_classes_to_take)

# Step 2: Copy TrashNet dataset (glass goes to the same 'glass' folder)
trashnet_classes_to_take = ['metal', 'paper', 'plastic', 'cardboard', 'trash', 'glass']
copy_images(trashnet_path, include_classes=trashnet_classes_to_take)

print("✅ Merged dataset created with single 'glass' class at:", merged_path)

✅ Merged dataset created with single 'glass' class at: merged_dataset


In [ ]:
import random
# Define how many images to remove per class
remove_counts = {
    'clothes': 4300,
    'glass': 1600,
    'shoes': 1000
}

for cls, n_remove in remove_counts.items():
    class_folder = merged_path / cls
    images = list(class_folder.glob('*'))  # all files in the folder
    print(f"{cls}: {len(images)} images before removal")

    if len(images) <= n_remove:
        print(f"Cannot remove {n_remove} images from {cls}, only {len(images)} available")
        continue

    to_remove = random.sample(images, n_remove)  # pick random files
    for f in to_remove:
        os.remove(f)

    print(f"{cls}: {len(list(class_folder.glob('*')))} images after removal")

clothes: 5325 images before removal
clothes: 1025 images after removal
glass: 2512 images before removal
glass: 912 images after removal
shoes: 1977 images before removal
shoes: 977 images after removal


In [ ]:
dataset = datasets.ImageFolder(root="./merged_dataset/")

class_counts = {cls: 0 for cls in dataset.classes}

for _,label in dataset:
  class_name = dataset.classes[label]
  class_counts[class_name]+=1

for cls, count in class_counts.items():
  print(f"{cls} : {count} images")

battery : 945 images
biological : 985 images
cardboard : 891 images
clothes : 1025 images
glass : 912 images
metal : 769 images
paper : 1050 images
plastic : 865 images
shoes : 977 images
trash : 697 images


In [ ]:
shutil.make_archive("dataset", 'zip', "./merged_dataset/")

# Download it
from google.colab import files
files.download("dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Transform (resize, normalize, data augmentation for only train)
# Pytorch image preprocessing pipeline using torchvision.transforms

# train pipline with data augumentation (apply flip, rotate and brightness/contrast changes to image)
train_transform = transforms.Compose([ #combine multiple transformations into one pipeline
    transforms.Resize((224,224)), # reize to 224x 224 pixels (Pretrained models like ResNet, VGG expect this size as input)
    # on-the-fly data augmentation
    transforms.RandomHorizontalFlip(), # randomly flips the image left to right
    transforms.RandomRotation(20), #randomly rotates the image by +- 20 degress
    transforms.ColorJitter(brightness=0.2, contrast =0.2), # randomly changes the brightness and contrast
    transforms.ToTensor(), # PIL image or numpy array into pytorch tensor
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]) # normalize each channel(R,G,B) -> equation -> normalized = pixel-mean/std, mean is [0.5,0.5,0.5] and std is [0.5,0.5,0.5]
]
)


# validataion pipline only resize/ convert to tensors and normalize
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]) # should be last after totensor()
])


In [ ]:
# how many classes are there to classify?
classes = dataset.classes
print("Classes:", classes)

Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']


In [ ]:
len(dataset)

9116

In [ ]:
len(classes)

10

In [ ]:
# Split the dataset into 70% train, 15% val, 15%testquest
train_size = int(0.7* len(dataset))
val_size = int(0.15* len(dataset))
test_size = len(dataset)- train_size - val_size

In [ ]:
train_size

6381

In [ ]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size,test_size])


In [ ]:
# tansform pipeline application
train_dataset.dataset.transform =train_transform
val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform= val_transform

In [ ]:
img, label = train_dataset[0]

In [ ]:
img.size()

torch.Size([3, 224, 224])

In [ ]:
len(train_dataset)

6381

In [ ]:
class_counts = {cls: 0 for cls in train_dataset.dataset.classes}

for _,label in train_dataset:
  class_name = train_dataset.dataset.classes[label]
  class_counts[class_name]+=1

for cls, count in class_counts.items():
  print(f"{cls} : {count} images")

battery : 653 images
biological : 696 images
cardboard : 606 images
clothes : 695 images
glass : 640 images
metal : 541 images
paper : 741 images
plastic : 606 images
shoes : 704 images
trash : 499 images


In [ ]:

# 2. Compute class weights (inverse of counts)
num_samples = len(train_dataset)
class_weights = {cls: num_samples/count for cls, count in class_counts.items()}

# 3. Assign a weight to each sample
sample_weights = [0] * num_samples
for idx, (_, label) in enumerate(train_dataset):
    class_name = train_dataset.dataset.classes[label]
    sample_weights[idx] = class_weights[class_name]

# 4. Create sampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)


In [ ]:
# Dataloader to load the data in pytorch
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
num_classes = len(classes)

In [ ]:
num_classes

10

In [ ]:
model = models.resnet18(pretrained=True)

# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Replace the final FC layer for our number of classes
num_classes = len(train_dataset.dataset.classes)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


# loss function or criterion -> measures how wrong the model’s predictions are compared to true labels.
loss_function = nn.CrossEntropyLoss() # handles logits, no softmax needed

# optimizer-> Adam optimizer - a type of gradient descent+ momentum + adaptive learning rate
# computes gradients of the loss w.r.t. model parameters (via loss.backward()) and
# updates the weights to minimize the loss.
# Uses momentum (remembers past gradients)

optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-4)  # only train FC layer

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 182MB/s]


In [ ]:
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")


Epoch 1/15 | Loss: 1.7419 | Acc: 0.4954
Epoch 2/15 | Loss: 1.0120 | Acc: 0.7234
Epoch 3/15 | Loss: 0.7614 | Acc: 0.7812
Epoch 4/15 | Loss: 0.6347 | Acc: 0.8129
Epoch 5/15 | Loss: 0.5577 | Acc: 0.8337
Epoch 6/15 | Loss: 0.5371 | Acc: 0.8340
Epoch 7/15 | Loss: 0.4938 | Acc: 0.8475
Epoch 8/15 | Loss: 0.4682 | Acc: 0.8582
Epoch 9/15 | Loss: 0.4489 | Acc: 0.8566
Epoch 10/15 | Loss: 0.4216 | Acc: 0.8685
Epoch 11/15 | Loss: 0.4110 | Acc: 0.8651
Epoch 12/15 | Loss: 0.3892 | Acc: 0.8737
Epoch 13/15 | Loss: 0.3854 | Acc: 0.8771
Epoch 14/15 | Loss: 0.3909 | Acc: 0.8784
Epoch 15/15 | Loss: 0.3511 | Acc: 0.8927


In [ ]:
model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:  # assuming you have test_loader
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.8977


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

In [ ]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
image_path = "bo.jpg"  # replace with your image
image = Image.open(image_path).convert("RGB")  # ensure 3 channels

# Apply transforms and add batch dimension
input_tensor = transform(image).unsqueeze(0)  # shape: (1, 3, 224, 224)
input_tensor = input_tensor.to(device)  # move to GPU if used

# -----------------------------
# 4. Predict
# -----------------------------
with torch.no_grad():
    output = model(input_tensor)           # logits
    _, predicted = torch.max(output, 1)    # predicted class index

# Map index to class name
class_names = train_dataset.dataset.classes  # same order as your dataset
predicted_class = class_names[predicted.item()]

print(f"Predicted class: {predicted_class}")

Predicted class: plastic


In [ ]:
torch.save(model.state_dict(), "trashnet_model.pth")


files.download("trashnet_model.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Output Size formula - `W - F + 2P)/S + 1 `**bold text**

- (224 - 3 +2)/1 +1 = 224
- (112 -3+2)/1 +1 = 112

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TrashNetCNN(nn.Module):
    def __init__(self, num_classes=6):
        super(TrashNetCNN, self).__init__()

        # --- Convolution Blocks ---
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.3)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.3)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.4)
        )

        self.conv_block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.4)
        )

        # --- Global Average Pooling instead of huge FC ---
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))  # output: (batch, 256,1,1)

        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)

        x = self.avgpool(x)        # (batch, 256,1,1)
        x = torch.flatten(x,1)     # (batch, 256)
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)            # logits, CrossEntropyLoss handles softmax
        return x



In [ ]:
# create the model

cnn= TrashNetCNN(num_classes=6)
print(cnn)

TrashNetCNN(
  (conv_block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.3, inplace=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.3, inplace=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
# loss function or criterion -> measures how wrong the model’s predictions are compared to true labels.
loss_function = nn.CrossEntropyLoss()

# optimizer-> Adam optimizer - a type of gradient descent+ momentum + adaptive learning rate
# computes gradients of the loss w.r.t. model parameters (via loss.backward()) and
# updates the weights to minimize the loss.
# Uses momentum (remembers past gradients)

optimizer = optim.Adam(cnn.parameters(), lr= 0.001)



In [ ]:
num_epochs = 23
cnn.to(device)
for epoch in range(num_epochs):
  print(f"Training epoch {epoch} ....")

  cnn.train() # set the cnn model to training mode
  running_loss =0.0  # to accumulate total loss for the epoch
  correct = 0  # to count correct predictions
  total = 0 # total number of samples
  for imgs,labels in train_loader:
    imgs, labels = imgs.to(device), labels.to(device) # move data to GPU

    optimizer.zero_grad() # reset gradients from previous step

    outputs = cnn(imgs) # forward pass
    loss = loss_function(outputs, labels) # compute loss -> output we got Vs actual desire output

    loss.backward() # back propagation ( no need to specify manually )
    optimizer.step() # update weights using optimizer

    running_loss += loss.item()

  train_loss = running_loss/len(train_loader.dataset)
  print(f"Loss:{train_loss:.4f}")


Training epoch 0 ....
Loss:0.0553
Training epoch 1 ....
Loss:0.0547
Training epoch 2 ....
Loss:0.0533
Training epoch 3 ....
Loss:0.0522
Training epoch 4 ....
Loss:0.0507
Training epoch 5 ....
Loss:0.0501
Training epoch 6 ....
Loss:0.0490
Training epoch 7 ....
Loss:0.0491
Training epoch 8 ....
Loss:0.0482
Training epoch 9 ....
Loss:0.0483
Training epoch 10 ....
Loss:0.0482
Training epoch 11 ....
Loss:0.0478
Training epoch 12 ....
Loss:0.0477
Training epoch 13 ....
Loss:0.0473
Training epoch 14 ....
Loss:0.0480
Training epoch 15 ....
Loss:0.0471
Training epoch 16 ....
Loss:0.0475
Training epoch 17 ....
Loss:0.0466
Training epoch 18 ....
Loss:0.0473
Training epoch 19 ....
Loss:0.0465
Training epoch 20 ....
Loss:0.0464
Training epoch 21 ....
Loss:0.0464
Training epoch 22 ....
Loss:0.0467


In [ ]:
# Save the model
torch.save(cnn.state_dict(),'trashNEt_cnn.pth')

In [ ]:
# Recreate the model structure
model = TrashNetCNN(num_classes=6)
model.load_state_dict(torch.load("trashNEt_cnn.pth"))
model.to(device)


TrashNetCNN(
  (conv_block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.3, inplace=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.3, inplace=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropo

In [ ]:
# Validation
# set the model in eval mode
model.eval()
val_loss = 0.0 # to get the total loss of validation dataset
val_correct =0 # to count correctly predicted samples
val_total = 0 # to count total samples

with torch.no_grad(): # no need to compute gradients during validation ( save memory and computation)
  for imgs, labels in val_loader:
    imgs, labels = imgs.to(device), labels.to(device) # move to device
    outputs = model(imgs) # forward pass: get model predictions

    loss = loss_function(outputs, labels) # compute the loss

    val_loss +=loss.item() *imgs.size(0) # accumulate total loss weighted by batch size
    # maximum of the ouput
    _, predicted = torch.max(outputs,1) # get predicted class
    val_total +=labels.size(0) # update total number of samples seen
    val_correct +=(predicted == labels).sum().item() # count how many predicted correctly -> if the predicted labels is equal to the desire labels then correct +1


# average loss
val_loss /= val_total
# accuracy over the entire set
val_accuracy = val_correct/val_total

print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy:{val_accuracy:.4f}")


Validation Loss: 1.5184, Validation Accuracy:0.3747


In [ ]:
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = loss_function(outputs, labels)

        test_loss += loss.item() * imgs.size(0)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_loss /= test_total
test_acc = test_correct / test_total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 1.4108, Test Accuracy: 0.4447
